# Investigating the Noise Floor as a Function of Temperature. 

In [5]:
%load_ext autoreload

In [6]:
%autoreload 2
import os 
import sys
import struct 
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})
import numpy as np
import scipy.interpolate as interp 
from scipy.stats import chi2

sys.path.append("/home/drew/He6CRES/he6-cres-spec-sims/")
from he6_cres_spec_sims.daq.frequency_domain_packet import FDpacket

In [3]:
## Investigating our noise floor with a colder Noise floor: 

## Creating a new noise file to match the data taken on 4/30/21: (10072021)


In [42]:
path = os.getcwd()
background_file_1 ="/home/drew/He6CRES/CRESAnalysisAndSimulation/SpecFiles/NoiseMeasurement_10152021/Freq_data_2021-10-15-16-31-24.spec"
background_file_2 = "/home/drew/He6CRES/CRESAnalysisAndSimulation/SpecFiles/NoiseMeasurement_10012021/Freq_data_2021-10-01-16-34-34.spec"
background_file_3 = "/home/drew/He6CRES/CRESAnalysisAndSimulation/SpecFiles/KrTest_04302021/I_side_LessCharged/Freq_data_2021-04-30-13-12-35.spec"
background_file_4 = "/home/drew/He6CRES/CRESAnalysisAndSimulation/SpecFiles/NoiseMeasurement_10212021/Freq_data_2021-10-21-15-10-24_000000.spec"
background_file_5 = "/home/drew/He6CRES/CRESAnalysisAndSimulation/SpecFiles/NoiseMeasurement_10212021/Freq_data_2021-10-21-15-58-08_000000.spec"
background_file_6 = "/home/drew/He6CRES/CRESAnalysisAndSimulation/SpecFiles/NoiseMeasurement_10212021/Freq_data_2021-10-21-16-15-07_000000.spec"
# Create the background to be altered.
freq_bins = 8192
slices_to_grab = 1000
packets_per_slice = 4
background_1 = np.zeros((slices_to_grab*packets_per_slice,freq_bins), dtype = np.uint8)
background_2 = np.zeros((slices_to_grab*packets_per_slice,freq_bins), dtype = np.uint8)
background_3 = np.zeros((slices_to_grab*packets_per_slice,freq_bins), dtype = np.uint8)
background_4 = np.zeros((slices_to_grab*packets_per_slice,freq_bins), dtype = np.uint8)
background_5 = np.zeros((slices_to_grab*packets_per_slice,freq_bins), dtype = np.uint8)
background_6 = np.zeros((slices_to_grab*packets_per_slice,freq_bins), dtype = np.uint8)

In [40]:
# open file 1: 
try: 
    with open(background_file_1,'rb') as in_file:
        print("Opened .spec noise file.")
        for m in range(slices_to_grab*packets_per_slice):
            c = in_file.read(FDpacket.BYTES_IN_PACKET)
            packet = FDpacket.from_byte_string(c)
            background_1[m] = packet.interpret_data()
        
except Exception as e:
    raise e
    

Opened .spec noise file.


In [11]:
# open file 1: 
try: 
    with open(background_file_2,'rb') as in_file:
        print("Opened .spec noise file.")
        for m in range(slices_to_grab*packets_per_slice):
            c = in_file.read(FDpacket.BYTES_IN_PACKET)
            packet = FDpacket.from_byte_string(c)
            background_2[m] = packet.interpret_data()
        
except Exception as e:
    raise e
    

Opened .spec noise file.


In [12]:
# open file 1: 
try: 
    with open(background_file_3,'rb') as in_file:
        print("Opened .spec noise file.")
        for m in range(slices_to_grab*packets_per_slice):
            c = in_file.read(FDpacket.BYTES_IN_PACKET)
            packet = FDpacket.from_byte_string(c)
            background_3[m] = packet.interpret_data()
        
except Exception as e:
    raise e

Opened .spec noise file.


In [34]:
# open file 1: 
try: 
    with open(background_file_4,'rb') as in_file:
        print("Opened .spec noise file.")
        for m in range(slices_to_grab*packets_per_slice):
            c = in_file.read(FDpacket.BYTES_IN_PACKET)
            packet = FDpacket.from_byte_string(c)
            background_4[m] = packet.interpret_data()
        
except Exception as e:
    raise e

Opened .spec noise file.


In [47]:
# open file 1: 
try: 
    with open(background_file_5,'rb') as in_file:
        print("Opened .spec noise file.")
        for m in range(slices_to_grab*packets_per_slice):
            c = in_file.read(FDpacket.BYTES_IN_PACKET)
            packet = FDpacket.from_byte_string(c)
            background_5[m] = packet.interpret_data()
        
except Exception as e:
    raise e

Opened .spec noise file.


In [43]:
# open file 1: 
try: 
    with open(background_file_6,'rb') as in_file:
        print("Opened .spec noise file.")
        for m in range(slices_to_grab*packets_per_slice):
            c = in_file.read(FDpacket.BYTES_IN_PACKET)
            packet = FDpacket.from_byte_string(c)
            background_6[m] = packet.interpret_data()
        
except Exception as e:
    raise e

Opened .spec noise file.


In [44]:
background_1_flattened = np.concatenate((background_1[0::4],background_1[1::4],background_1[2::4],background_1[3::4]), axis =1)
background_2_flattened = np.concatenate((background_2[0::4],background_2[1::4],background_2[2::4],background_2[3::4]), axis =1)
background_3_flattened = np.concatenate((background_3[0::4],background_3[1::4],background_3[2::4],background_3[3::4]), axis =1)
background_4_flattened = np.concatenate((background_4[0::4],background_4[1::4],background_4[2::4],background_4[3::4]), axis =1)
background_5_flattened = np.concatenate((background_5[0::4],background_5[1::4],background_5[2::4],background_5[3::4]), axis =1)
background_6_flattened = np.concatenate((background_6[0::4],background_6[1::4],background_6[2::4],background_6[3::4]), axis =1)


pts_in_spline = 200
dev = int(freq_bins*4/pts_in_spline)
print("Select every {}th element in the background_flattened array to do the spline. \n".format(dev))

noise_floor_1 = background_1_flattened.mean(axis = 0)
noise_floor_2 = background_2_flattened.mean(axis = 0)
noise_floor_3 = background_3_flattened.mean(axis = 0)
noise_floor_4 = background_4_flattened.mean(axis = 0)
noise_floor_5 = background_5_flattened.mean(axis = 0)
noise_floor_6 = background_6_flattened.mean(axis = 0)

noise_floor_bins = np.arange(0,noise_floor_1.shape[0],1)

# Conduct a 300 pt cubic spline fit
noise_floor_cubicspline_1 = interp.CubicSpline(noise_floor_bins[::dev], noise_floor_1[::dev], extrapolate = True)
noise_floor_cubicspline_2 = interp.CubicSpline(noise_floor_bins[::dev], noise_floor_2[::dev], extrapolate = True)
noise_floor_cubicspline_3 = interp.CubicSpline(noise_floor_bins[::dev], noise_floor_3[::dev], extrapolate = True)
noise_floor_cubicspline_4 = interp.CubicSpline(noise_floor_bins[::dev], noise_floor_4[::dev], extrapolate = True)
noise_floor_cubicspline_5 = interp.CubicSpline(noise_floor_bins[::dev], noise_floor_5[::dev], extrapolate = True)
noise_floor_cubicspline_6 = interp.CubicSpline(noise_floor_bins[::dev], noise_floor_6[::dev], extrapolate = True)


noise_floor_smooth_1 = noise_floor_cubicspline_1(noise_floor_bins).clip(0)
noise_floor_smooth_2 = noise_floor_cubicspline_2(noise_floor_bins).clip(0)
noise_floor_smooth_3 = noise_floor_cubicspline_3(noise_floor_bins).clip(0)
noise_floor_smooth_4 = noise_floor_cubicspline_4(noise_floor_bins).clip(0)
noise_floor_smooth_5 = noise_floor_cubicspline_5(noise_floor_bins).clip(0)
noise_floor_smooth_6 = noise_floor_cubicspline_6(noise_floor_bins).clip(0)

Select every 163th element in the background_flattened array to do the spline. 



In [49]:
%matplotlib qt
noise_floor_freq = noise_floor_bins*1200/noise_floor_bins.shape[0]

# Conduct a 300 pt cubic spline fit
# plt.plot(noise_floor_freq,noise_floor_smooth_1/8.0, label = "Decay Cell @ 52K, Atten @ 8K")
# plt.plot(noise_floor_freq,noise_floor_smooth_2/8.0, label = "Decay Cell @ 52K, Atten @ ~20K")
# plt.plot(noise_floor_freq,noise_floor_smooth_4/8.0, label = "Decay Cell @ 76K, Atten @ 8K")
plt.plot(noise_floor_freq,noise_floor_smooth_5/8.0, label = "U-Side, Decay Cell @ 85K, Atten @ 8K")
plt.plot(noise_floor_freq,noise_floor_smooth_6/8.0, label = "U-Side, Decay Cell @ 97K, Atten @ 8K")

# plt.plot(noise_floor_freq,noise_floor_smooth_3, label = "Decay Cell @ 133K, Atten @ ~20K")

plt.title("Noise Floor (requant gain = 17)")
plt.xlabel("Freq (MHz)")
plt.ylabel("Power (arb.)")
plt.legend()
plt.show()

In [63]:
difference = (noise_floor_smooth_150K - noise_floor_smooth_50K/8)/(noise_floor_smooth_50K/8)
# diff_2 = (noise_floor_smooth_150K/(noise_floor_smooth_50K/8))
plt.plot(noise_floor_freq,difference)
plt.plot(noise_floor_freq,np.ones_like(noise_floor_bins)*difference[2731:].mean(), label = "Mean Relative Diff = {}$\%$ "
         .format(np.around(difference[2731:].mean(),3)*100 ))
plt.title("Relative Difference in Noise Power (requant gain = 17)")
plt.xlabel("Freq (MHz)")
plt.ylabel("Power (arb.)")
plt.show()
plt.legend()
# print(diff_2[2731:].mean())

inf


/home/drew/home/drew/Products/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  
/home/drew/home/drew/Products/anaconda3/lib/python3.7/site-packages/ipykernel/eventloops.py:106: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  app.exec_()


In [ ]:
background_1 = np.zeros((slices_to_grab*packets_per_slice,freq_bins), dtype = np.uint8)

In [24]:
## Could the LNAs be a limiting factor in the reduction of our noise floor?

T_e = 6 # K
k_b = 1.3807e-23 # joule/K
B = 36.6e3 

N = k_b*T_e*B
print("Noise Power from LNA's:" , N*1e15, "fW")

Noise Power from LNA's: 0.0030320171999999998 fW
